## Low Level Look at the Basics of NLP 
#### This is a notebook based on the NLP chapter in "Data Science from Scratch" by xxx. Please check out it out here  

In [2]:
# !pip install beautifulsoup4

In [4]:
import re
from bs4 import BeautifulSoup
import requests

url = "https://www.oreilly.com/ideas/what-is-data-science"
html = requests.get(url).text
soup = BeautifulSoup(html, 'html5lib')

In [10]:
def fix_unicode(text: str) -> str:
    return text.replace(u"\u2019", "'")

In [5]:
soup

<!DOCTYPE html>
<html lang="en-US"><head>

    <meta charset="utf-8"/>

    <title>
      What is data science? â OâReilly     </title>

    <link href="https://www.oreilly.com/radar/wp-content/themes/odot/style.css" rel="stylesheet"/>

    
    <link href="//s.w.org" rel="dns-prefetch"/>
		<script type="text/javascript">
			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/12.0.0-1\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/12.0.0-1\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/www.oreilly.com\/radar\/wp-includes\/js\/wp-emoji-release.min.js?ver=5.3.2"}};
			!function(e,a,t){var r,n,o,i,p=a.createElement("canvas"),s=p.getContext&&p.getContext("2d");function c(e,t){var a=String.fromCharCode;s.clearRect(0,0,p.width,p.height),s.fillText(a.apply(this,e),0,0);var r=p.toDataURL();return s.clearRect(0,0,p.width,p.height),s.fillText(a.apply(this,t),0,0),r===p.toDataURL()}function l(e){if(!s||!s.fillText)return!1;switch

In [8]:
content = soup.find("div", "main-post-radar-content")   # find article-body div
content

<div class="main-post-radar-content">

              
              
              
              
              
              <p>Weâve all heard it: according to Hal Varian, <a href="http://www.nytimes.com/2009/08/06/technology/06stats.html">statistics is the next sexy job</a>. Five years ago, in <a href="http://oreilly.com/web2/archive/what-is-web-20.html">What is Web 2.0</a>, Tim OâReilly said that âdata is the next Intel Inside.â But what does that statement mean? Why do we suddenly care about statistics and about data?</p>
<p>In this post, I examine the many sides of data science â the technologies, the companies and the unique skill sets.</p><div class="zjsXkrQb"><div class="inline-cta trial-cta" id="trial-cta" itemscope="" itemtype="http://schema.org/Product">
  <div class="thumb">
    <a href="https://www.oreilly.com/online-learning/">
      <img class="" itemprop="image" src="https://d3ansictanv2wj.cloudfront.net/safari-topic-cta-1f60e6f96856da19ba3cb25660472ca5.jpg

In [11]:
regex = r"[\w']+|[\.]"                       # matches a word or a period
document = []

for paragraph in content("p"):
    words = re.findall(regex, fix_unicode(paragraph.text))
    document.extend(words)

In [13]:
document[:10]

['Weâ',
 've',
 'all',
 'heard',
 'it',
 'according',
 'to',
 'Hal',
 'Varian',
 'statistics']